# Converting LDA topic words into human readable topics using LLMs via DSPy

The output of LDA topic modeling yields on list of words as topics which are not directly human readable. In this notebook, we will use large language models (LLMs) to generate proper topic names. In particular, we will use the DSPy framework and OpenAI models.

See also:
- DSPy: The framework for programming—not prompting—language models https://dspy.ai/

## Dependencies

```bash
pip install dspy azure-identity>=1.19.0 python-dotenv
```

In [19]:
import dspy
import pandas as pd

In [ ]:
# The topics from `topics.ipynb` (reading the LDA output directly would be better!)
topics = """Topic 1:
Top words: maustaste, wenn sie, kopieren, linke, halten, gedrückt, markieren, halten sie, einfügen, sie markieren

Topic 2:
Top words: daten, bundesnetzagentur, ermittlung, beschwerdeführerin, enwg, festlegung, rahmen, generellen, vorgaben, berechnung

Topic 3:
Top words: dass sich, mündlichen, mündlichen verhandlung, verhandlung, vortrag, dann, ihrer, mehr, zwar, verhalten

Topic 4:
Top words: ifsg, corona, covid, personen, cov, sars, sars cov, maßnahmen, coronavirus, rki

Topic 5:
Top words: vwgo, verwaltungsgericht, verwaltungsgerichts, ovg, frage, gkg, beschwerde, kostenentscheidung, bedeutung, bverwg

Topic 6:
Top words: vwgo, wirkung, anordnung, vollziehung, interesse, behörde, aufschiebende, aufschiebende wirkung, vwvfg, verwaltungsakt

Topic 7:
Top words: beigeladenen, tätigkeit, beamten, beigeladene, bverwg, beurteilung, ovg, auswahlentscheidung, bewerber, dienstherrn

Topic 8:
Top words: straße, grundstück, beigeladenen, vorhaben, nutzung, baugenehmigung, genehmigung, gebäude, anlage, grundstücks

Topic 9:
Top words: bgb, bgh, zpo, olg, anspruch, aufl, njw, parteien, rechtsprechung, dann

Topic 10:
Top words: gutachten, behandlung, erkrankung, sachverständigen, stellungnahme, sachverständige, untersuchung, patienten, ärztlichen, medizinischen

Topic 11:
Top words: aufenthg, asylg, abschiebung, bundesamt, ausländer, deutschland, schutz, bverwg, einreise, emrk

Topic 12:
Top words: sinne, gesetzgeber, auslegung, vorschrift, regelungen, anwendung, wortlaut, vorschriften, voraussetzungen, zweck

Topic 13:
Top words: bfh, estg, erteilung, kinder, gmbh, aufenthaltserlaubnis, mutter, kind, eltern, bstbl

Topic 14:
Top words: werde, schreiben, beantragt, hätten, begründung, würden, zur begründung, kosten, weiteren, zudem

Topic 15:
Top words: beschwerde, vwgo, innerhalb, westfalen, dokument, nordrhein, nordrhein westfalen, oberverwaltungsgericht, schriftlich, land

Topic 16:
Top words: eur, euro, kosten, jahr, betrag, zeitraum, insgesamt, berechnung, jeweils, aufwendungen

Topic 17:
Top words: baugb, gemeinde, bebauungsplan, bebauungsplans, satzung, planung, bverwg, festsetzungen, festsetzung, belange

Topic 18:
Top words: sgb, bsg, sgg, leistungen, anspruch, versicherten, lsg, sgb viii, viii, xii

Topic 19:
Top words: the, personen, situation, afghanistan, and, lage, report, rückkehr, auskunft, jahr

Topic 20:
Top words: verfolgung, niedersächsischen, niedersächsischen corona, nds, sinne, bverwg, corona, senatsbeschl, wahrscheinlichkeit, ovg

Topic 21:
Top words: anordnung, vwgo, einstweiligen, hauptsache, einstweiligen anordnung, erlass, glaubhaft, einstweilige, erlass einstweiligen, rechtsschutzes

Topic 22:
Top words: beschl, urt, vgh, baden, württemberg, baden württemberg, bverwg, vgh baden, bverfg, ovg

Topic 23:
Top words: stpo, beschwerde, stgb, olg, amtsgericht, angeklagten, europäischen, aufl, angeklagte, eugh

Topic 24:
Top words: bverfg, bvr, öffentlichen, ihrer, anderen, personen, recht, interesse, schutz, versammlung

Topic 25:
Top words: kündigung, parteien, seit, anlage, arbeitnehmer, bag, azr, zinsen, nebst, bgb
"""

# very hacky way of parsing the string input
topic_lines = topics.splitlines()
topic_words = [topic_lines[i+1][len("Top words: "):].split(", ") for i in range(len(topic_lines)) if (i % 3) == 0]
topic_id_to_words = {i: words for i, words in enumerate(topic_words, 1)}
pd.DataFrame(topic_id_to_words)

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
0,maustaste,daten,dass sich,ifsg,vwgo,vwgo,beigeladenen,straße,bgb,gutachten,...,eur,baugb,sgb,the,verfolgung,anordnung,beschl,stpo,bverfg,kündigung
1,wenn sie,bundesnetzagentur,mündlichen,corona,verwaltungsgericht,wirkung,tätigkeit,grundstück,bgh,behandlung,...,euro,gemeinde,bsg,personen,niedersächsischen,vwgo,urt,beschwerde,bvr,parteien
2,kopieren,ermittlung,mündlichen verhandlung,covid,verwaltungsgerichts,anordnung,beamten,beigeladenen,zpo,erkrankung,...,kosten,bebauungsplan,sgg,situation,niedersächsischen corona,einstweiligen,vgh,stgb,öffentlichen,seit
3,linke,beschwerdeführerin,verhandlung,personen,ovg,vollziehung,beigeladene,vorhaben,olg,sachverständigen,...,jahr,bebauungsplans,leistungen,afghanistan,nds,hauptsache,baden,olg,ihrer,anlage
4,halten,enwg,vortrag,cov,frage,interesse,bverwg,nutzung,anspruch,stellungnahme,...,betrag,satzung,anspruch,and,sinne,einstweiligen anordnung,württemberg,amtsgericht,anderen,arbeitnehmer
5,gedrückt,festlegung,dann,sars,gkg,behörde,beurteilung,baugenehmigung,aufl,sachverständige,...,zeitraum,planung,versicherten,lage,bverwg,erlass,baden württemberg,angeklagten,personen,bag
6,markieren,rahmen,ihrer,sars cov,beschwerde,aufschiebende,ovg,genehmigung,njw,untersuchung,...,insgesamt,bverwg,lsg,report,corona,glaubhaft,bverwg,europäischen,recht,azr
7,halten sie,generellen,mehr,maßnahmen,kostenentscheidung,aufschiebende wirkung,auswahlentscheidung,gebäude,parteien,patienten,...,berechnung,festsetzungen,sgb viii,rückkehr,senatsbeschl,einstweilige,vgh baden,aufl,interesse,zinsen
8,einfügen,vorgaben,zwar,coronavirus,bedeutung,vwvfg,bewerber,anlage,rechtsprechung,ärztlichen,...,jeweils,festsetzung,viii,auskunft,wahrscheinlichkeit,erlass einstweiligen,bverfg,angeklagte,schutz,nebst
9,sie markieren,berechnung,verhalten,rki,bverwg,verwaltungsakt,dienstherrn,grundstücks,dann,medizinischen,...,aufwendungen,belange,xii,jahr,ovg,rechtsschutzes,ovg,eugh,versammlung,bgb


In [ ]:
# Setup LLM
import os
import dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

dotenv.load_dotenv()  # load variables from .env file

# Replace these variables with your own settings
# (for example you can run a local LLM: https://dspy.ai/#__tabbed_1_5)
model_name = "azure/GPT-4o"
model_api_version = "2024-12-01-preview"
model_api_base_url = os.environ["AZURE_OPENAI_API_BASE_URL"]
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)()  # this is only needed if you use Azure

lm = dspy.LM(
    model=model_name,
    api_version=model_api_version,
    api_base=model_api_base_url,
    azure_ad_token=token_provider,
)
dspy.configure(lm=lm)

In [27]:
class TopicNameGenerator(dspy.Signature):
    """Generate a human readable name for each list of topic words (output of LDA topic modeling)."""

    topic_id_to_words: dict[int, list[str]] = dspy.InputField(desc="Topic ID => topic words")
    topic_to_to_name: dict[int, str] = dspy.OutputField(desc="Topic ID => name")


In [ ]:
# Call the DSPy program
topic_generator = dspy.Predict(TopicNameGenerator)
prediction = topic_generator(
    topic_id_to_words=topic_id_to_words,
)
prediction.topic_to_to_name  # {1: 'Text Editing and Copying', 2: 'Data and Regulations', ... 


{1: 'Text Editing and Copying',
 2: 'Data and Regulations',
 3: 'Oral Proceedings and Behavior',
 4: 'COVID-19 Measures',
 5: 'Administrative Court Procedures',
 6: 'Administrative Actions and Effects',
 7: 'Employment and Selection',
 8: 'Property and Construction',
 9: 'Legal Claims and Judgments',
 10: 'Medical Assessments',
 11: 'Immigration and Asylum',
 12: 'Legislation and Interpretation',
 13: 'Tax and Residency',
 14: 'Application and Justification',
 15: 'Complaints and Documentation',
 16: 'Financial Calculations',
 17: 'Urban Planning',
 18: 'Social Security and Benefits',
 19: 'Afghanistan Situation Reports',
 20: 'COVID-19 Legal Pursuits',
 21: 'Interim Orders',
 22: 'Judicial Decisions',
 23: 'Criminal Proceedings',
 24: 'Public Rights and Interests',
 25: 'Employment Termination'}